In [1]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import utils 
import torch
from torch import nn
from torchsummary import summary

import pytorch_lightning as pl



In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
data = utils.SynthData.spiral()

In [25]:
class EncoderLayer(nn.Module):
    def __init__(self, nFeature=32):
        super(EncoderLayer, self).__init__()
        self.conv = nn.Conv1d(nFeature, nFeature, kernel_size=5, dilation=3 ,padding='same')
        self.batchNorm = nn.BatchNorm1d(nFeature)
        self.selu = nn.SELU()
        self.batchNorm2 = nn.BatchNorm1d(nFeature)
                
    def forward(self, x):
        res = x
        x = self.conv(x)
        x = self.batchNorm(x)
        x = self.selu(x)
        
        res = self.batchNorm2(res)
        x += res
        return x

class Encoder(nn.Module):
    def __init__(self, nLayer=16, nFeature=32):
        super(Encoder, self).__init__()
        self.seq = nn.Sequential()
        for i in range(nLayer):
            self.seq.add_module(f'conv{i}', EncoderLayer(nFeature=nFeature))
    
    def forward(self, x):
        x = self.seq(x)
        return x

    



class TrajNet(pl.LightningModule):
    def __init__(self, nhidden=8, nLayer=16, nFeature=32):
        super(TrajNet, self).__init__()
        self.conv_enc = nn.Conv1d(2, nFeature, kernel_size=1)
        self.encoder = Encoder(nLayer=nLayer, nFeature=nFeature)
        self.mu = nn.Linear(nFeature, nhidden)
        self.log_var = nn.Linear(nFeature, nhidden)
    
    @staticmethod
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample        
        
    def forward(self, x):
        # x: (batch_size, 2, time_step)
        x = self.conv_enc(x)
        x = self.encoder(x)
        x = x[:, :, 0]
        x = x.flatten(1)
        mu = self.mu(x)
        log_var = self.log_var(x)
        
        
        return mu, log_var
    
# model = TrajNet()
# d = torch.rand(5, 2, 3)
# d = model.forward(d)
# d



RuntimeError: The size of tensor a (15) must match the size of tensor b (3) at non-singleton dimension 2

In [28]:

class DecoderLayer(nn.Module):
    def __init__(self, nFeature=32):
        super(DecoderLayer, self).__init__()
        self.convT = nn.ConvTranspose1d(nFeature, nFeature, kernel_size=5, dilation=3)
        self.batchNorm1 = nn.BatchNorm1d(nFeature)
        self.selu = nn.SELU()
        self.batchNorm2 = nn.BatchNorm1d(nFeature)
        
    def forward(self, x):
        res = x
        x = self.convT(x)
        x = self.batchNorm1(x)
        x = self.selu(x)
        # res = self.batchNorm2(res)
        # x += res
        return x
    
model = DecoderLayer()
model(torch.rand(5, 32, 3)).shape

torch.Size([5, 32, 15])

In [37]:
l = nn.ConvTranspose1d(3, 3, kernel_size=5, dilation=3)
x = torch.rand(1, 3, 1)
x = x.repeat(1, 1, 10)
l(x)[0, 0, :].shape

torch.Size([22])